In [9]:
import holoviews
import panel
import xarray

import pandas
import geoviews

import echogram

In [7]:
MVBS_ds = xarray.open_mfdataset(
    paths = 'concatenated_MVBS.nc', 
    data_vars = 'minimal', coords='minimal',
    combine = 'by_coords'
)
MVBS_ds

<xarray.Dataset>
Dimensions:            (channel: 4, echo_range: 150, ping_time: 875)
Coordinates:
  * channel            (channel) object 'GPT  18 kHz 009072058c8d 1-1 ES18-11...
  * echo_range         (echo_range) float64 0.0 5.0 10.0 ... 735.0 740.0 745.0
  * ping_time          (ping_time) datetime64[ns] 2017-07-24T19:30:00 ... 201...
    time1              (ping_time) datetime64[ns] dask.array<chunksize=(875,), meta=np.ndarray>
Data variables:
    Sv                 (channel, ping_time, echo_range) float64 dask.array<chunksize=(4, 875, 150), meta=np.ndarray>
    frequency_nominal  (channel) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude          (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
    latitude           (ping_time) float64 dask.array<chunksize=(875,), meta=np.ndarray>
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.7.1
    processing_time:              2023-05-30T17:40:45Z
    processing_function:          commongrid.compute_MVBS

In [45]:
import numpy

opt_lines=geoviews.opts(width=600, 
                        height=400, 
                        color="red", 
                        tools=["hover", "box_select"], 
                        line_width=1)

opt_start_point=geoviews.opts(color="red", 
                              size=8)

opt_point=geoviews.opts(color="blue", 
                        tools=["hover"], 
                        size=10)

def plot_tiles(map_tiles="OSM"):
    """
    Plot map tiles using GeoViews.

    Parameters:
        map_tiles (str, optional): The type of map tiles to plot.
            Defaults to "OSM". 
            See more in : https://holoviews.org/reference/elements/bokeh/Tiles.html

    Returns:
        geoviews.element: Map tiles object from GeoViews representing the specified map tiles.
    """
    tiles = getattr(geoviews.tile_sources, map_tiles)

    return tiles

def plot_track(
    MVBS_ds,
    opts_line=opt_lines,
    opts_point=opt_start_point,
):
    """
    Plot a track using GeoViews.

    Parameters
    ----------
    MVBS_ds (xarray.Dataset): 
        MVBS Dataset with coordinates 'ping_time', 'channel', 'echo_range'.
        MVBS Dataset with values 'latitue', 'longtitue'.

    opts_line (geoviews.opts, optional): Options for styling the track line.

    opts_line (geoviews.opts, optional): Options for styling the starting point.

    Returns
    -------
    holoviews.Overlay: Combined chart consisting of the ship track lines and starting point.
    """

    # convert xarray data to geoviews data
    all_pd_data = pandas.concat(
        [
            pandas.DataFrame(MVBS_ds.longitude.values, columns=["Longitude"]),
            pandas.DataFrame(MVBS_ds.latitude.values, columns=["Latitude"]),
            pandas.DataFrame(MVBS_ds.ping_time.values, columns=["Ping Time"]),
        ],
        axis=1,
    )

    all_pd_data = all_pd_data.dropna(axis=0, how="any")

    # plot path
    line = geoviews.Path(
        [all_pd_data],
        kdims=["Longitude", "Latitude"],
        vdims=["Ping Time", "Longitude", "Latitude"],
    ).opts(opts_line)

    # plot start node
    start_data = all_pd_data.iloc[0].values.tolist()

    start_node = geoviews.Points(
        [start_data],
        kdims=["Longitude", "Latitude"],
    ).opts(opts_point)

    return line * start_node


def plot_point(
    MVBS_ds,
    opts_point=opt_point,
):
    """

    Parameters
    ----------
    MVBS_ds : xarray.Dataset
        MVBS Dataset with coordinates 'ping_time', 'channel', 'echo_range'
        MVBS Dataset has been combined with longitude & latitude coordinates using echopype

    opts_line : geoviews.opts

    opts_point : geoviews.opts

    map_tiles : str, default : 'OSM'
        See more in : https://holoviews.org/reference/elements/bokeh/Tiles.html

    Returns
    -------
    holoviews.Overlay
        Combined chart(track, start point and map tile)

    """

    # plot start node
    # index = get_first_non_null_index(MVBS_ds.longitude.values)

    point = geoviews.Points(
        [MVBS_ds.longitude.values[0], 
         MVBS_ds.latitude.values[0], 
         MVBS_ds.ping_time.values[0]],
        kdims=["Longitude", "Latitude"],
    ).opts(opts_point)

    return point


def get_first_non_null_index(data):
    """
    Get the first index of non-null data from a one-dimensional NumPy ndarray.

    Parameters:
        data (numpy.ndarray): The input one-dimensional ndarray.

    Returns:
        int or None: The index of the first non-null value from the ndarray, or None if no non-null value is found.
    """
    mask = numpy.isnan(data)  # Create boolean mask for null values
    if numpy.all(mask):  # Check if all values are null
        return None
    return numpy.argmax(~mask)  # Find index of first non-null value using argmax

In [46]:
plot_point(MVBS_ds)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[int64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>)

:Points   [Longitude,Latitude]

In [41]:
help(geoviews.opts.Points)

Help on method builder in module holoviews.util:

builder(*, active_tools, align, alpha, angle, apply_extents, apply_ranges, aspect, autorange, axiswise, backend, bgcolor, border, cformatter, clabel, clim, clim_percentile, clipping_colors, cmap, cnorm, color, color_levels, colorbar, colorbar_opts, colorbar_position, data_aspect, default_span, default_tools, fill_alpha, fill_color, fixed_bounds, fontscale, fontsize, frame_height, frame_width, framewise, global_extent, gridstyle, height, hooks, hover_alpha, hover_color, hover_fill_alpha, hover_fill_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_join, hover_line_width, infer_projection, invert_axes, invert_xaxis, invert_yaxis, jitter, labelled, legend_cols, legend_labels, legend_muted, legend_offset, legend_opts, legend_position, line_alpha, line_cap, line_color, line_dash, line_join, line_width, lod, logx, logy, logz, margin, marker, max_height, max_width, min_height, min_width, muted, muted_alpha,